In [66]:
import urllib
from urllib.error import HTTPError
import requests
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import csv
from xml import etree
from xml.sax.saxutils import unescape
import xml.etree.ElementTree as ET
import json
import html
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import unicodedata
import time
import re

In [67]:
google_base_url = "https://maps.googleapis.com/maps/api/geocode/json?address="
auth_token = (env.token)

In [68]:
def get_root(url):
    req = urllib.request.Request(url=url)
    try:
        handler = urllib.request.urlopen(req)
        tree = ET.ElementTree(file = handler)
        root = tree.getroot()
        return root
    except HTTPError as e:
        content = e.read()
        return content

In [69]:
base_url = "https://www.costa-coffee.cz/kavarny.php"

In [70]:
root = get_root(base_url)                        

In [71]:
def get_column_names(root):
    tags = []
    for i, child in enumerate(root):  
        for subchild in child[-1:]: 
            for k in subchild:
                tags.append(k.tag)
    return tags              

In [72]:
tags = get_column_names(root)

In [73]:
tags

['title',
 'description',
 'pubDate',
 'category',
 'image',
 'link',
 'guid',
 'wifi',
 'kuracka_zona',
 'bezbarierovy_pristup',
 'detsky_koutek',
 'oteviraci_doba']

In [74]:
def get_data(root):
    records = []
    data = []
    for i, child in enumerate(root):
        for subchild in child:
            for j in subchild:
                records.append(j.text)
            try:
                if len(records) != 0:
                    data.append(records)
                    records = []
            except AttributeError as error:
                return error
    return data

In [75]:
records = get_data(root)

In [ ]:
records

In [77]:
rawDf = pd.DataFrame(records, columns=tags)

In [79]:
def unescape(x):
        return (html.unescape(x).encode("utf-8", "xmlcharrefreplace").decode())

In [80]:
unescapeDf = rawDf
unescapeDf["description"] = unescapeDf["description"].apply(unescape)

In [82]:
# maze nepotrebne sloupce
unescapeDf.drop(unescapeDf.columns[[2,3, 5, 6]], axis=1, inplace=True)

In [84]:
# prejmenuje sloupce
renameUnescapeDf = unescapeDf.rename(columns={'description': 'address', 'kuracka_zona': 'smoking_zone', 'bezbarierovy_pristup': 'accessible', 'detsky_koutek': 'playground','oteviraci_doba': 'opening_hours'})

In [85]:
def make_request(address, google_base_url, auth_token):
        url = google_base_url + urllib.parse.quote(address) + auth_token
        try:
            response = requests.get(url)
            response_json = json.loads(response.text)
            return response_json
        except response == "":
            response_json = NaN
        except requests.exceptions.HTTPError as errh:
            print ("Http Error:",errh)
        except requests.exceptions.ConnectionError as errc:
            print ("Error Connecting:",errc)
        except requests.exceptions.Timeout as errt:
            print ("Timeout Error:",errt)
        except requests.exceptions.RequestException as err:
            print ("OOps: Something Else",err)

In [86]:
def get_coordinates(address):
    response = make_request(address, google_base_url, auth_token)
    result = {}
    lat = str(round(response["results"][0]["geometry"]["location"]["lat"],6))
    lng = str(round(response["results"][0]["geometry"]["location"]["lng"],6))
    result = {"lat":lat, "lng":lng}
    return result

In [87]:
def sanitize_html(value):
    value = value.strip()
    value = value.replace('\n','')
    value = value.replace('\t','')
    return re.sub(r'<[^>]*?>', '', value)

In [88]:
def coordinates(row):
    address_coordinates = get_coordinates(row["address"])
    row["lat"] = address_coordinates["lat"]
    row["lng"] = address_coordinates["lng"]
    return row

In [89]:
sanitizedAddressDf = renameUnescapeDf
sanitizedAddressDf["address"] = sanitizedAddressDf["address"].apply(sanitize_html)

In [90]:
sanitizedTimesDf = sanitizedAddressDf
sanitizedTimesDf["opening_hours"] = sanitizedTimesDf["opening_hours"].apply(sanitize_html)


In [91]:
coordinatesDf = sanitizedTimesDf
coordinatesDf = coordinatesDf.apply(coordinates, axis=1)

In [92]:
YesNoDf = coordinatesDf

In [93]:
def yes_no_recode(value):
    recoded = np.where(value == "Ano",1,0)
    return recoded

In [94]:
YesNoDf["wifi"] = YesNoDf["wifi"].apply(yes_no_recode)
YesNoDf["smoking_zone"] = YesNoDf["smoking_zone"].apply(yes_no_recode)
YesNoDf["accessible"] = YesNoDf["accessible"].apply(yes_no_recode)
YesNoDf["playground"] = YesNoDf["playground"].apply(yes_no_recode)

In [96]:
def df_to_json(df):
    feed_json = df.to_json(orient='records')
    temp_po = json.loads(feed_json, encoding='utf-8')
    temp_json = json.dumps(temp_po, ensure_ascii = False)
    json_no_space = temp_json.replace('\\', '')
    json_no_space = temp_json.replace('\n', ' ')
    return temp_json

In [97]:
feed_json = df_to_json(YesNoDf)

In [98]:
def print_json(json):
    return print(json)

In [ ]:
print_json(feed_json)